# **Installations**

In [ ]:
!nvidia-smi
# Uncomment only if u have enough GPU? FOR FASTER execution

In [ ]:
import subprocess

# Download the wkhtmltopdf .deb package
subprocess.run(["wget", "-q", "https://github.com/wkhtmltopdf/packaging/releases/download/0.12.6-1/wkhtmltox_0.12.6-1.bionic_amd64.deb"])

# Install the downloaded package
subprocess.run(["dpkg", "-i", "wkhtmltox_0.12.6-1.bionic_amd64.deb"])

# Fix any dependencies if needed
subprocess.run(["apt-get", "install", "-f"])


CompletedProcess(args=['apt-get', 'install', '-f'], returncode=0)

In [ ]:
!apt-get update
!apt-get upgrade

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [52.9 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,035 kB]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,370 kB]
Hit:13 https://ppa.launchp

In [ ]:
!apt-get --fix-broken install

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


In [ ]:
!pip install -qqq transformers==4.27.2 pytorch-lightning==1.9.4 torchmetrics==0.11.4 imgkit==1.2.3 easyocr==1.6.2 Pillow==9.4.0 tensorboardX==2.5.1 huggingface_hub==0.11.1 gdown protobuf~=3.19 --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 kB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.

In [ ]:
from transformers import LayoutLMv3FeatureExtractor, LayoutLMv3TokenizerFast, LayoutLMv3Processor, LayoutLMv3ForSequenceClassification
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from sklearn.model_selection import train_test_split
import imgkit
import easyocr
import torchvision.transforms as T
from pathlib import Path
import matplotlib.pyplot as plt
import os
import cv2
from typing import List
import json
from torchmetrics import Accuracy
from huggingface_hub import notebook_login
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

%matplotlib inline
pl.seed_everything(42)

INFO:lightning_fabric.utilities.seed:Global seed set to 42


42

In [ ]:
!pip install pdf2image
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 4 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.3 [186 kB]
Fetched 186 kB in 0s (2,224 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121925 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.3_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.3) ...
Setting up poppler-utils (22.02.0-2ubuntu0.3) ...
Processing triggers for man-db (2.10.2-1) ...


# **Load Trained Model**

In [ ]:
model = LayoutLMv3ForSequenceClassification.from_pretrained('EmnaFazaa/layoutlmv3-financial-document-classification')


Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/504M [00:00<?, ?B/s]

# **Load Financial report & convert it to images**

In [ ]:
# from pdf2image import convert_from_path

# pdf_path = '/content/drive/MyDrive/MyDrive/MyFiles/Data/amen_bank_efd311222.pdf'
# image_folder = '/content/drive/MyDrive/MyDrive/MyFiles/Data/images'

# pages = convert_from_path(pdf_path, 300)  # Set the DPI (dots per inch) as needed

# for i, page in enumerate(pages):
#     page.save(f'{image_folder}page_{i + 1}.png', 'PNG')


## **Initialize EasyOCR Reader and Load Image**




*   Use an OCR (Optical Character Recognition) tool to extract text from the image.
*   Obtain bounding box information for each detected word or text region.
*   Combine the extracted text and bounding box information into a format suitable for input to the model.




In [ ]:
!pip install easyocr

In [ ]:
from PIL import Image
import easyocr

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Load the image
image_path = "/content/drive/MyDrive/MyDrive/MyFiles/Data/CashFlowEng.png"
image = Image.open(image_path).convert("RGB")

# Perform OCR on the image
ocr_result = reader.readtext(str(image_path))
print(ocr_result)


In [ ]:
from pathlib import Path
import cv2
from PIL import Image, ImageDraw, ImageFont

font_path = Path(cv2.__path__[0]) / "qt/fonts/DejaVuSansCondensed.ttf"
print(font_path.exists())
font = ImageFont.truetype(str(font_path), size=12)

In [ ]:
def create_bounding_box(bbox_data):
    xs = []
    ys = []
    for x, y in bbox_data:
        xs.append(x)
        ys.append(y)

    left = int(min(xs))
    top = int(min(ys))
    right = int(max(xs))
    bottom = int(max(ys))

    return [left, top, right, bottom]

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(28, 28))

left_image = Image.open(image_path).convert("RGB")
right_image = Image.new("RGB", left_image.size, (255, 255, 255))

left_draw = ImageDraw.Draw(left_image)
right_draw = ImageDraw.Draw(right_image)

for i, (bbox, word, confidence) in enumerate(ocr_result):
    box = create_bounding_box(bbox)

    left_draw.rectangle(box, outline="blue", width=2)
    left, top, right, bottom = box

    left_draw.text((right + 5, top), text=str(i + 1), fill="red", font=font)
    right_draw.text((left, top), text=word, fill="black", font=font)

ax1.imshow(left_image)
ax2.imshow(right_image)
ax1.axis("off");
ax2.axis("off");

Saving the OCR results in JSON format

In [ ]:
from pathlib import Path

# Convert image_path to a Path object
image_path = Path(image_path)
ocr_result = reader.readtext(str(image_path), batch_size=16)

ocr_page = []
for bbox, word, confidence in ocr_result:
    ocr_page.append({
        "word": word, "bounding_box": create_bounding_box(bbox)
    })

with image_path.with_suffix(".json").open("w") as f:
    json.dump(ocr_page, f)


# **Preprocessing**


*  Load the JPEG image directly using an image library like PIL or OpenCV.
*  Resize the image to a suitable size required by your model.
*   Resize the image to a suitable size required by your model.

In [ ]:
feature_extractor = LayoutLMv3FeatureExtractor(apply_ocr=False)
tokenizer = LayoutLMv3TokenizerFast.from_pretrained("microsoft/layoutlmv3-base")
processor = LayoutLMv3Processor(feature_extractor, tokenizer)

In [ ]:
image = Image.open(image_path).convert("RGB")
width, height = image.size
width, height

In [ ]:
width_scale = 1000 / width
height_scale = 1000 / height

width_scale, height_scale

In [ ]:
image_path=Path(image_path)
json_path = image_path.with_suffix(".json")
json_path

In [ ]:
with json_path.open("r") as f:
    ocr_result = json.load(f)

In [ ]:
def scale_bounding_box(box: List[int], width_scale : float = 1.0, height_scale : float = 1.0) -> List[int]:
    return [
        int(box[0] * width_scale),
        int(box[1] * height_scale),
        int(box[2] * width_scale),
        int(box[3] * height_scale)
    ]

In [ ]:
words = []
boxes = []
for row in ocr_result:
    boxes.append(scale_bounding_box(row["bounding_box"], width_scale, height_scale))
    words.append(row["word"])

In [ ]:
len(words), len(boxes)

In [ ]:
encoding = processor(
    image,
    words,
    boxes=boxes,
    max_length=512,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

encoding.keys()

In [ ]:
print(f"""
input_ids:  {list(encoding["input_ids"].squeeze().shape)}
word boxes: {list(encoding["bbox"].squeeze().shape)}
image data: {list(encoding["pixel_values"].squeeze().shape)}
image size: {image.size}
""")

**image**

In [ ]:
image_data = encoding["pixel_values"][0]
image_data.shape

In [ ]:
transform = T.ToPILImage()
transform(image_data)

**Word boxes**

In [ ]:
def unnormalize_box(bbox, width, height):
    return [
        width * (bbox[0] / 1000),
        height * (bbox[1] / 1000),
        width * (bbox[2] / 1000),
        height * (bbox[3] / 1000),
    ]

In [ ]:
input_ids=encoding["input_ids"].flatten(),
attention_mask=encoding["attention_mask"].flatten(),
bbox=encoding["bbox"].flatten(end_dim=1),
pixel_values=encoding["pixel_values"].flatten(end_dim=1),
#labels=torch.tensor(label, dtype=torch.long)

# **All in one fct: preprocessing + Classification**

In [ ]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
def predict_document_image(
    image_path: Path,
    model: LayoutLMv3ForSequenceClassification,
    processor: LayoutLMv3Processor):

    json_path = image_path.with_suffix(".json")
    with json_path.open("r") as f:
        ocr_result = json.load(f)

        with Image.open(image_path).convert("RGB") as image:

            width, height = image.size
            width_scale = 1000 / width
            height_scale = 1000 / height

            words = []
            boxes = []
            for row in ocr_result:
                boxes.append(
                    scale_bounding_box(
                        row["bounding_box"],
                        width_scale,
                        height_scale
                    )
                )
                words.append(row["word"])

            encoding = processor(
                image,
                words,
                boxes=boxes,
                max_length=512,
                padding="max_length",
                truncation=True,
                return_tensors="pt"
            )

    with torch.inference_mode():
        output = model(
            input_ids=encoding["input_ids"].to(DEVICE),
            attention_mask=encoding["attention_mask"].to(DEVICE),
            bbox=encoding["bbox"].to(DEVICE),
            pixel_values=encoding["pixel_values"].to(DEVICE)
        )

    predicted_class = output.logits.argmax()
    return model.config.id2label[predicted_class.item()]

# **Perform Inference**
Prepare input tensors including token IDs for the text, attention mask, scaled bounding box coordinates, and possibly pixel values of the resized image.

In [ ]:
labels = []
predictions = []
labels.append(image_path.parent.name)
predictions.append(predict_document_image(image_path, model, processor))

In [ ]:
predictions